In [3]:
import random
import json
import pickle
import numpy as np
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
nltk.download('punkt')
with open("/content/sample_data/intents.json", encoding='utf-8') as file:
    data = json.load(file)

texts = []
labels = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        texts.append(pattern.lower())
        labels.append(intent['tag'])

# Etiketleri sayıya çevir
tag_to_index = {tag: i for i, tag in enumerate(set(labels))}
index_to_tag = {i: tag for tag, i in tag_to_index.items()}
numeric_labels = [tag_to_index[label] for label in labels]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# One-hot encode labels
one_hot_labels = to_categorical(numeric_labels)


In [10]:
  from nltk.stem import WordNetLemmatizer
  lemmatizer = WordNetLemmatizer()
try:
    with open("/content/sample_data/intents.json", encoding='latin-1') as f:
        intents = json.load(f)
except UnicodeDecodeError:
    try:
        with open("/content/sample_data/intents.json", encoding='cp1252') as f:
            intents = json.load(f)
    except UnicodeDecodeError:
        print("Could not decode the file with either 'latin-1' or 'cp1252' encoding. Please check the file's encoding.")
        raise

words = []
classes = []
documents = []
ignore_letters = ['?','!','.','.']
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

In [11]:
print(intents.keys())

dict_keys(['intents'])


In [12]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern.lower())
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]  # Added lower()
words = sorted(set(words))
classes = sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [13]:
print(words)

["'m", "'re", "'s", '911', 'a', 'about', 'abroad', 'academic', 'accident', 'active', 'activity', 'adios', 'adopting', 'adventure', 'advice', 'affair', 'afternoon', 'age', 'aid', 'alert', 'alright', 'am', 'amazing', 'ambition', 'ambivalent', 'an', 'analysis', 'and', 'angry', 'animal', 'annoying', 'anxiety', 'anxious', 'any', 'anyone', 'appreciate', 'appreciated', 'apps', 'are', 'artist', 'ask', 'asleep', 'assist', 'assistance', 'at', 'attorney', 'auto', 'average', 'awake', 'awesome', 'awful', 'baby', 'bad', 'baking', 'balanced', 'be', 'beauty', 'become', 'beginner', 'behavior', 'being', 'bender', 'best', 'better', 'big', 'bill', 'birth', 'birthday', 'blessed', 'block', 'board', 'book', 'born', 'bos', 'bot', 'brain', 'breakdown', 'breaking', 'breakup', 'briefing', 'brilliant', 'bring', 'broke', 'budget', 'budgeting', 'bunch', 'business', 'but', 'buy', 'buying', 'by', 'bye', 'ca', 'call', 'calm', 'can', 'capability', 'car', 'cardio', 'care', 'career', 'cash', 'cat', 'catch', 'challenge', 

In [14]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_len))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tag_to_index), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.fit(padded_sequences, one_hot_labels, epochs=200, batch_size=8, verbose=1)
model.save("chatbot_cnn_model.h5")


Epoch 1/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0165 - loss: 4.0627
Epoch 2/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0480 - loss: 4.0326
Epoch 3/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1132 - loss: 3.7917
Epoch 4/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2251 - loss: 3.0288
Epoch 5/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3931 - loss: 2.2623
Epoch 6/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5095 - loss: 1.7014
Epoch 7/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6593 - loss: 1.2724
Epoch 8/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7299 - loss: 0.9764
Epoch 9/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7797 - loss: 0.8150
Epoch 10/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8321 - loss: 0.6208
Epoch 11/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8575 - loss: 0.5054
Epoch 12/200
143/143 ━━━━━━━━━━━━━━━━━━━━

In [16]:
def predict_intent(sentence):
    seq = tokenizer.texts_to_sequences([sentence.lower()])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    preds = model.predict(padded)[0]
    intent_index = np.argmax(preds)
    intent = index_to_tag[intent_index]
    confidence = preds[intent_index]
    return intent, confidence

def get_response(intent, intents_data):
    for intent_data in intents_data['intents']:
        if intent_data['tag'] == intent:
            return random.choice(intent_data['responses'])
    return "Can you repeat?"


In [17]:
while True:
    msg = input("You: ")
    if msg.lower() == "exit":
        break
    intent, conf = predict_intent(msg)
    response = get_response(intent, data)
    print(f"Bot ({intent} - %{conf*100:.1f}):", response)


You: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Bot (greeting - %100.0): Greetings! How may I assist you?
You:  how are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Bot (greeting - %100.0): Hello! How can I help you today?
You: who are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot (name_query - %100.0): They call me ChatBot! Nice to meet you.
You: i want to happy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Bot (health - %52.3): Take care of yourself! When in doubt about health matters, it's best to consult a doctor.
You: i am sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Bot (technology - %58.6): Check cables and connections first when having tech troubles. Many issues are simple fixes.
You: thank you, goodbye!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot (thanks - %100.0): Anytime! That's what I'm here for.
You: exit


In [19]:
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [22]:
# Giriş boyutları

# Etiketleri sayısallaştır ve encode et
y = [tag_to_index[label] for label in labels]
y_encoded = to_categorical(y)

input_dim = len(tokenizer.word_index) + 1
output_dim = 50
input_length = max_len
num_classes = y_encoded.shape[1]

In [23]:
# RNN modeli
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim, output_dim, input_length=input_length))
rnn_model.add(SimpleRNN(64))
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dropout(0.5))
rnn_model.add(Dense(num_classes, activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
print("\n🔄 RNN modeli eğitiliyor...")
rnn_model.fit(padded_sequences, y_encoded, epochs=200, batch_size=8, verbose=1)
rnn_acc = rnn_model.evaluate(padded_sequences, y_encoded, verbose=0)[1]
print(f"✅ RNN doğruluk: %{rnn_acc * 100:.2f}")


🔄 RNN modeli eğitiliyor...
Epoch 1/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0117 - loss: 4.0604
Epoch 2/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0801 - loss: 3.7784
Epoch 3/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2168 - loss: 2.9883
Epoch 4/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3936 - loss: 2.2438
Epoch 5/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5404 - loss: 1.7287
Epoch 6/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6380 - loss: 1.3303
Epoch 7/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6727 - loss: 1.2104
Epoch 8/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7388 - loss: 0.9905
Epoch 9/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7927 - loss: 0.8055
Epoch 10/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8052 - loss: 0.7262
Epoch 11/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8325 - loss: 0.6371
Epoch 12/200


In [27]:
def predict_with_rnn(sentence):
    input_data = preprocess_input(sentence)
    pred = rnn_model.predict(input_data)[0]
    index = np.argmax(pred)
    intent = index_to_tag[index]
    confidence = pred[index]
    print(f"🤖 RNN → Tahmin: {intent} (%{confidence*100:.2f})")

In [29]:
def preprocess_input(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    sequence = tokenizer.texts_to_sequences([" ".join(lemmatized)])
    padded_input = pad_sequences(sequence, maxlen=max_len, padding='post')
    return padded_input
while True:
    msg = input("✉️  Bir cümle yaz (çıkmak için 'çıkış'): ")
    if msg.lower() == "çıkış":
        break
    predict_with_rnn(msg)
    print("—" * 50)

✉️  Bir cümle yaz (çıkmak için 'çıkış'): Hi!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
🤖 RNN → Tahmin: greeting (%100.00)
——————————————————————————————————————————————————
✉️  Bir cümle yaz (çıkmak için 'çıkış'): How Are You?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
🤖 RNN → Tahmin: greeting (%100.00)
——————————————————————————————————————————————————
✉️  Bir cümle yaz (çıkmak için 'çıkış'): Who are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
🤖 RNN → Tahmin: name_query (%100.00)
——————————————————————————————————————————————————
✉️  Bir cümle yaz (çıkmak için 'çıkış'): i want happy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
🤖 RNN → Tahmin: stress (%99.96)
——————————————————————————————————————————————————
✉️  Bir cümle yaz (çıkmak için 'çıkış'): i want sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
🤖 RNN → Tahmin: stress (%98.88)
——————————————————————————————————————————————————
✉️  Bir cümle yaz (çıkmak için 'çıkış'): çıkış


In [32]:
# BiLSTM modeli
bilstm_model = Sequential()
bilstm_model.add(Embedding(input_dim, output_dim, input_length=input_length))
bilstm_model.add(Bidirectional(LSTM(64)))
bilstm_model.add(Dense(64, activation='relu'))
bilstm_model.add(Dropout(0.5))
bilstm_model.add(Dense(num_classes, activation='softmax'))
bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("\n🔄 BiLSTM modeli eğitiliyor...")
bilstm_model.fit(padded_sequences, y_encoded, epochs=200, batch_size=8, verbose=1)
bilstm_acc = bilstm_model.evaluate(padded_sequences, y_encoded, verbose=0)[1]
print(f"✅ BiLSTM doğruluk: %{bilstm_acc * 100:.2f}")



🔄 BiLSTM modeli eğitiliyor...
Epoch 1/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0120 - loss: 4.0622
Epoch 2/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0322 - loss: 4.0474
Epoch 3/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0625 - loss: 3.7468
Epoch 4/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1577 - loss: 3.1198
Epoch 5/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2814 - loss: 2.4705
Epoch 6/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4460 - loss: 1.8597
Epoch 7/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5463 - loss: 1.5313
Epoch 8/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6490 - loss: 1.1360
Epoch 9/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7533 - loss: 0.8603
Epoch 10/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7753 - loss: 0.7324
Epoch 11/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8212 - loss: 0.5985
Epoch 12/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
